##### This notebook exports the updated blacklist based on HH user annotations, along with the IDs of optimization articles and the value counts needed for data_count.xlsx.

Before running the notebook, please ensure the following files are located in the `data/01_raw/annotated_contents` directory:

- `user_annotation_25jul.xlsx`: Contains annotated remarks on the 'User Annotation (To Combine)' tab provided by the HH team.
- `Stage 1 user annotation for HPB_HHcomments_20Aug24.xlsx`: Contains annotated remarks on the 'User Annotation (To Optimize)' tab provided by the HH team.
- `final_predicted_clusters (first iteration).xlsx`: The first iteration of clustering results is required since the results may change with each iteration.

In [1]:
import os

import pandas as pd

In [2]:
%load_ext kedro.ipython

[08/28/24 16:28:15] INFO     Using                                                                  ]8;id=10654;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=263854;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\framework\project\__init__.py#246\246]8;;\
                             'c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\                
                             kedro\framework\project\rich_logging.yml' as logging configuration.                   

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=571191;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=252631;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py#58\58]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=51155;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=317112;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py#60\60]8;;\

                    INFO     Resolved project path as:                                              ]8;id=643825;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=228844;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py#175\175]8;;\
                             c:\Users\Sixia\healthhub-content-optimization\content-optimization.                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[08/28/24 16:28:18] INFO     Kedro project Content Optimization                                     ]8;id=168915;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=33444;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=839762;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=379917;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py#142\142]8;;\
                             'pipelines'                                                                           

[08/28/24 16:28:21] INFO     Registered line magic 'run_viz'                                        ]8;id=991603;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=692169;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py#148\148]8;;\

In [3]:
export_folder_path = r"..\data\08_reporting"
input_folder_path = r"..\data\01_raw\annotated_contents"

### Get IDs for optimisation
- Take into account HH user annotation in `Stage 1 user annotation for HPB_HHcomments_20Aug24.xlsx`
- Remove all articles with the comment 'exclude-XXX' in `HH's team's comments` column

In [4]:
user_annotation = pd.read_excel(
    input_folder_path + "/Stage 1 user annotation for HPB_HHcomments_20Aug24.xlsx",
    sheet_name=3,
)
for_optimisation = user_annotation[
    ~user_annotation["HH's team's comments"].fillna("").str.contains("exclude")
]

for_optimisation_id = for_optimisation[["article_id"]]
print("\nNumber of articles for optimisation: ", len(for_optimisation))
for_optimisation_value_counts = (
    for_optimisation["content category"]
    .value_counts()
    .reset_index()
    .sort_values("content category")
)
print(for_optimisation_value_counts)  # to update data_count.xlsx

export_path = os.path.join(export_folder_path, "ids_for_optimisation.csv")
for_optimisation_id.to_csv(export_path, index=False)

[08/28/24 16:28:24] WARNING  c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\o ]8;id=349653;file://C:\Users\Sixia\AppData\Local\Programs\Python\Python310\lib\warnings.py\warnings.py]8;;\:]8;id=892164;file://C:\Users\Sixia\AppData\Local\Programs\Python\Python310\lib\warnings.py#109\109]8;;\
                             penpyxl\worksheet\_reader.py:329: UserWarning: Data Validation                        
                             extension is not supported and will be removed                                        
                               warn(msg)                                                                           
                                                                                                                   


Number of articles for optimisation:  244
              content category  count
2           cost-and-financing      1
1      diseases-and-conditions     24
0        live-healthy-articles    218
3  medical-care-and-facilities      1


### Get updated blacklist

1. Blacklist articles labelled as 'exclude' in `Stage 1 user annotation for HPB_HHcomments_20Aug24.xlsx`
2. Blacklist 'Nasi Liwet with Brown Rice' article with 'recipe' as the reason

In [5]:
# Load current articles that are blacklisted

initial_blacklist = catalog.load("params:blacklist")  # noqa: F821
initial_blacklist_df = pd.DataFrame(
    list(initial_blacklist.items()), columns=["article_id", "remove_type"]
)

# Concat initial blacklist with new blacklist
to_exclude = user_annotation[
    user_annotation["HH's team's comments"].fillna("").str.contains("exclude")
]
to_exclude = to_exclude[["article_id", "HH's team's comments"]]
to_exclude.columns = ["article_id", "remove_type"]
print("No. of articles to add on for blacklist: ", len(to_exclude))
to_exclude["remove_type"] = to_exclude["remove_type"].str.replace("exclude-", "")
to_exclude["remove_type"] = to_exclude["remove_type"].apply(lambda x: x.title())
to_exclude["remove_type"] = to_exclude["remove_type"].str.replace(
    "No Relevant Content And Mainly Link", "No relevant content and mainly links"
)
to_exclude["remove_type"] = to_exclude["remove_type"].str.replace(
    "Table Of Contents", "Table of Contents"
)
blacklist_update = pd.concat([initial_blacklist_df, to_exclude])

# Get article id of nasi liwet
add_on_title_to_blacklist = ["Nasi Liwet with Brown Rice"]
add_on_blacklist_lower = [title.lower() for title in add_on_title_to_blacklist]
merged_data = catalog.load("merged_data")  # noqa: F821
add_on_blacklist_id_title = merged_data[
    merged_data["title"]
    .str.lower()
    .apply(
        lambda x: any(
            blacklist_title in x for blacklist_title in add_on_blacklist_lower
        )
    )
][["id", "title"]]
print(add_on_blacklist_id_title)

# Label the blacklist reason
added_blacklist = {1445657: "Recipe"}
added_blacklist_df = pd.DataFrame(
    list(added_blacklist.items()), columns=["article_id", "remove_type"]
)

# Update blacklist
blacklist_update_2 = pd.concat([blacklist_update, added_blacklist_df])
blacklist_update_2.sort_values("remove_type", inplace=True)
blacklist_update_2.drop_duplicates("article_id", inplace=True)

# Write new blacklist into a txt file
export_path = os.path.join(export_folder_path, "blacklist_update.txt")
with open(export_path, "w") as file:
    for i, row in blacklist_update_2.iterrows():
        file.write(f"{row['article_id']}: \"{row['remove_type']}\"\n")

print("\nNumber of articles in blacklist:", blacklist_update_2.shape[0])

                    INFO     Loading data from params:blacklist (MemoryDataset)...              ]8;id=964427;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=331334;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\io\data_catalog.py#537\537]8;;\

No. of articles to add on for blacklist:  32


                    INFO     Loading data from merged_data (ParquetDataset)...                  ]8;id=830533;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=251823;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\io\data_catalog.py#537\537]8;;\

           id                       title
1505  1445657  Nasi Liwet with Brown Rice

Number of articles in blacklist: 42


### Before proceeding:

1. Paste the updated blacklist (`blacklist_update.txt`) into `parameters_data_processing.yml`
2. Run `kedro run --pipeline=data_processing` to obtain the updated `merged_data.parquet`

In [6]:
%reload_kedro

                    INFO     Resolved project path as:                                              ]8;id=69272;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=45364;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py#175\175]8;;\
                             C:\Users\Sixia\healthhub-content-optimization\content-optimization.                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[08/28/24 16:28:26] INFO     Kedro project Content Optimization                                     ]8;id=966534;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=608963;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=193139;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=576625;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py#142\142]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=987856;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=769623;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\ipython\__init__.py#148\148]8;;\

### Get Excluded Articles Tab and Data Count

In [22]:
merged_data = catalog.load("merged_data")  # noqa: F821
merged_data_HPB = merged_data[
    merged_data["pr_name"].fillna("").str.contains("Health Promotion Board")
]
to_keep_content_category = [
    "cost-and-financing",
    "diseases-and-conditions",
    "live-healthy-articles",
    "medical-care-and-facilities",
    "support-group-and-others",
]
merged_data_HPB_cat = merged_data_HPB[
    merged_data_HPB["content_category"].isin(to_keep_content_category)
]
print(merged_data_HPB_cat.shape)

print(
    "\nValue Counts Before Filtering: \n"
)  # data_count.xlsx - 'HPB raw data count in excel'
hpb_raw_value_counts = (
    merged_data_HPB_cat.content_category.value_counts()
    .reset_index()
    .sort_values("content_category")
)
print(hpb_raw_value_counts)
to_remove = merged_data_HPB_cat[merged_data_HPB_cat["to_remove"]]
to_remove_value_counts = (
    to_remove.content_category.value_counts()
    .reset_index()
    .sort_values("content_category")
)

print(
    "\nValue Counts of to_remove articles:", len(to_remove), "\n"
)  # data_count.xlsx - 'to_remove articles (final)'
print(to_remove_value_counts)


## Exclude tab to exclude those duplicated URL and content articles (considered backend issues)
additional_exclusion_from_excluded_tab = [
    1444417,
    1445629,
    1445972,
    1445829,
]  # 1445829 - sayur lodeh tagged as recipe but is also duplicated
exclude_tab = to_remove[
    [
        "id",
        "title",
        "full_url",
        "content_category",
        "page_views",
        "article_category_names",
        "remove_type",
    ]
]
exclude_tab = exclude_tab[~exclude_tab.id.isin(additional_exclusion_from_excluded_tab)]
exclude_tab_value_counts = (
    exclude_tab.content_category.value_counts()
    .reset_index()
    .sort_values("content_category")
)
hpb_raw_value_counts_exclude_duplicated = merged_data_HPB_cat[
    ~merged_data_HPB_cat.id.isin(additional_exclusion_from_excluded_tab)
].content_category.value_counts()
hpb_raw_value_counts_exclude_duplicated = (
    pd.DataFrame(hpb_raw_value_counts_exclude_duplicated)
    .sort_values("content_category")
    .reset_index()
)
print(
    "\n\nValue Counts Before Filtering (Exclude Duplicated):\n",
    hpb_raw_value_counts_exclude_duplicated,
)
print("\n\nValue Counts of to_remove articles (Exclude Duplicated):")
print(exclude_tab_value_counts)
export_path = os.path.join(export_folder_path, "excluded_content_tab.csv")

exclude_tab = exclude_tab[['id','title','full_url','content_category','article_category_names','page_views','remove_type']]
exclude_tab.rename(columns={'id':'article_id','full_url':'url','remove_type':'removal_reason'}, inplace=True)
exclude_tab.to_csv(export_path, index=False)

[08/28/24 17:05:49] INFO     Loading data from merged_data (ParquetDataset)...                  ]8;id=905994;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=291341;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\io\data_catalog.py#537\537]8;;\

(704, 39)

Value Counts Before Filtering: 

              content_category  count
3           cost-and-financing      1
1      diseases-and-conditions     42
0        live-healthy-articles    659
2  medical-care-and-facilities      2

Value Counts of to_remove articles: 76 

          content_category  count
1  diseases-and-conditions      3
0    live-healthy-articles     73


,id,remove_type
657,1444417,Duplicated URL
1067,1445629,Duplicated URL
1370,1445829,Recipe
1515,1445972,Duplicated Content




Value Counts Before Filtering (Exclude Duplicated):
               content_category  count
0           cost-and-financing      1
1      diseases-and-conditions     42
2        live-healthy-articles    655
3  medical-care-and-facilities      2


Value Counts of to_remove articles (Exclude Duplicated):
          content_category  count
1  diseases-and-conditions      3
0    live-healthy-articles     69


In [8]:
# To update in data_count.xlsx - 'HPB only to_remove articles Counts (final)'
remove_type_value_counts = (
    pd.DataFrame(to_remove.remove_type.value_counts())
    .sort_values("remove_type")
    .reset_index()
)
remove_type_value_counts

# Add sum row
total_sum = remove_type_value_counts["count"].sum()
total_row = pd.DataFrame([["Total", total_sum]], columns=["remove_type", "count"])
remove_type_article_counts_df = pd.concat(
    [remove_type_value_counts, total_row], ignore_index=True
)
print(remove_type_article_counts_df)

                                         remove_type  count
0                                 Duplicated Content      1
1                                     Duplicated URL      2
2                                        Infographic     14
3  LLM is unable to process content due to Azure'...      1
4               No relevant content and mainly links      6
5                                             Recipe     45
6                                 Services Directory      3
7                                  Table of Contents      4
8                                              Total     76


### Sent for Clustering Data Count

In [9]:
filtered_data_with_keywords = catalog.load("filtered_data_with_keywords")  # noqa: F821
print("No. of articles sent for clustering:", len(filtered_data_with_keywords))
cluster_value_counts = (
    filtered_data_with_keywords.content_category.value_counts()
    .reset_index()
    .sort_values("content_category")
)
print(cluster_value_counts)

[08/28/24 16:28:27] INFO     Loading data from filtered_data_with_keywords (ParquetDataset)...  ]8;id=9842;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=854837;file://c:\Users\Sixia\healthhub-content-optimization\venv\lib\site-packages\kedro\io\data_catalog.py#537\537]8;;\

No. of articles sent for clustering: 628
              content_category  count
3           cost-and-financing      1
1      diseases-and-conditions     39
0        live-healthy-articles    586
2  medical-care-and-facilities      2


### (To Combine) Data Count
- NOTE: This uses final_predicted_clusters from the first iteration that is sent to HH for annotation

In [10]:
annotation = pd.read_excel(
    input_folder_path + "/user_annotation_25jul.xlsx", sheet_name=1
)
annotation_indiv = annotation[
    (annotation["Action"] == "Individual") & (annotation["Algorithm remarks"].notna())
]

first_cluster_result = pd.read_excel(
    input_folder_path + "/final_predicted_clusters (first iteration).xlsx"
)
first_cluster_result_cluster = first_cluster_result[
    first_cluster_result["group_keywords"].notna()
]

# Remove those annotated by HH to be individual articles
first_cluster_result_cluster = first_cluster_result_cluster[
    ~first_cluster_result_cluster["id"].isin(annotation_indiv.article_id)
]

# Remove those in blacklist
blacklisted_articles_id = blacklist_update_2["article_id"].to_list()
first_cluster_result_cluster = first_cluster_result_cluster[
    ~first_cluster_result_cluster["id"].isin(blacklisted_articles_id)
]

first_cluster_result_cluster_id = first_cluster_result_cluster["id"].to_list()
whitelist_id = 1446081
first_cluster_result_cluster_id.append(whitelist_id)
first_cluster_result_cluster_w_cat = merged_data_HPB_cat[merged_data_HPB_cat.id.isin(first_cluster_result_cluster_id)][["id", "content_category"]]
to_combine_value_counts = (
    first_cluster_result_cluster_w_cat["content_category"]
    .value_counts()
    .reset_index()
    .sort_values("content_category")
)

### Consolidate Data Count

In [11]:
hpb_raw_value_counts.columns = ["content_category", "HPB raw data count in excel"]
hpb_raw_value_counts_exclude_duplicated.columns = [
    "content_category",
    "HPB raw data count in excel (Excludes Duplicated)",
]
to_remove_value_counts.columns = ["content_category", "to_remove articles"]
exclude_tab_value_counts.columns = [
    "content_category",
    "to_remove articles (Exclude Duplicated)",
]
cluster_value_counts.columns = ["content_category", "Sent for Clustering"]
to_combine_value_counts.columns = [
    "content_category",
    "Articles in clusters identified",
]
for_optimisation_value_counts.columns = [
    "content_category",
    "individual for optimisation",
]

In [12]:
combined_df = pd.concat(
    [
        hpb_raw_value_counts.set_index("content_category"),
        hpb_raw_value_counts_exclude_duplicated.set_index("content_category"),
        to_remove_value_counts.set_index("content_category"),
        exclude_tab_value_counts.set_index("content_category"),
        cluster_value_counts.set_index("content_category"),
        to_combine_value_counts.set_index("content_category"),
        for_optimisation_value_counts.set_index("content_category"),
    ],
    axis=1,
    join="outer",
).reset_index()

# Add on the content category 'support group and others' as all are zeros
support_grp = {col: 0 for col in combined_df.columns if col != "content_category"}
support_grp["content_category"] = "support group and others"
new_row = pd.DataFrame([support_grp])
combined_df = pd.concat([combined_df, new_row], ignore_index=True)


# Calculate the sum for each numeric column
sum_row = combined_df.sum(numeric_only=True)
sum_row["content_category"] = "Total"
sum_row_df = pd.DataFrame(sum_row).T
combined_df = pd.concat([combined_df, sum_row_df], ignore_index=True)

combined_df["content_category"] = combined_df["content_category"].str.replace("-", " ")
combined_df = combined_df.fillna(0)

[08/28/24 16:28:29] WARNING  C:\Users\Sixia\AppData\Local\Temp\ipykernel_23988\4018932954.py:29:    ]8;id=166295;file://C:\Users\Sixia\AppData\Local\Programs\Python\Python310\lib\warnings.py\warnings.py]8;;\:]8;id=513023;file://C:\Users\Sixia\AppData\Local\Programs\Python\Python310\lib\warnings.py#109\109]8;;\
                             FutureWarning: Downcasting object dtype arrays on .fillna, .ffill,                    
                             .bfill is deprecated and will change in a future version. Call                        
                             result.infer_objects(copy=False) instead. To opt-in to the future                     
                             behavior, set `pd.set_option('future.no_silent_downcasting', True)`                   
                               combined_df = combined_df.fillna(0)                                                 
                                                                                                                   

In [13]:
combined_df.to_csv(export_folder_path + "/content_category_data_count.csv", index=False)
remove_type_article_counts_df.to_csv(
    export_folder_path + "/remove_type_data_count.csv", index=False
)